# Загрузка датасетов и библиотек

In [ ]:
!wget https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/train.jsonl
!wget  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/val.jsonl
!wget  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/test.jsonl

In [ ]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  25580      0 --:--:-- --:--:-- --:--:-- 25580


In [ ]:
!pip install -U transformers
!pip install jsonlines
!pip install googletrans==4.0.0-rc1

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

from tqdm.notebook import tqdm

import numpy as np
import json
import pandas as pd
import json
import jsonlines

In [ ]:


size = 50000
batch_size = 4

def get_X_y(data_json_file):
    X, y = [], []
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            for i in range(0,500):
              text = text.replace('('+str(i)+')','')
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    for i in range(0,500):
                       a['text'] = a['text'].replace('('+str(i)+')','')
                    sep = text+' [QRY] '+query+' [ANS] '+a['text'] # for first block tarin uncomment
                    X.append([sep])
                    if data_json_file != 'test.jsonl':
                      y.append(a['label'])
                    
    return X, y

X_train, y_train = get_X_y('train.jsonl')
X_val, y_val = get_X_y('val.jsonl')

In [ ]:
with open('X_aug_train.txt', 'w') as filehandle:
    for listitem in X_aug_train:
        filehandle.write('%s\n' % listitem)
with open('y_aug_train.txt', 'w') as filehandle:
    for listitem in y_aug_train:
        filehandle.write('%s\n' % listitem)

In [ ]:
X_aug_train = []
y_aug_train = []
with open('X_aug_train.txt', 'r') as filehandle:
    for line in filehandle:
        currentPlace = line[:-1]
        X_aug_train.append(eval(currentPlace))

with open('y_aug_train.txt', 'r') as filehandle:
    for line in filehandle:
        currentPlace = line[:-1]
        y_aug_train.append(eval(currentPlace))

# Аугментация 

In [ ]:
!wget 'http://vectors.nlpl.eu/repository/20/212.zip'

--2021-01-09 23:50:20--  http://vectors.nlpl.eu/repository/20/212.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1635573390 (1.5G) [application/zip]
Saving to: ‘212.zip’

212.zip             100%[===================>]   1.52G  12.4MB/s    in 1m 55s  

2021-01-09 23:52:15 (13.6 MB/s) - ‘212.zip’ saved [1635573390/1635573390]



In [ ]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/20/212.zip'

with zipfile.ZipFile('212.zip', 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [ ]:
morph = pymorphy2.MorphAnalyzer(lang='ru')

In [ ]:
from googletrans import Translator

def tags_in_model(word):
    p = nltk.pos_tag([word])[0][1]
    if 'JJ' in p:
      return word + '_ADJ', '_ADJ'
    if 'VB' in p:
      return word + '_VERB', '_VERB'
    if 'RB' in p:
      return word + '_ADV', '_ADV'
    else:
      for i in ['_NOUN','_ADJ','_VERB']:
        if word + i in model:
          return word + i, i
    return 0,0

  
def similar(word_1, word_2):
    for i in model.most_similar(positive=[word], topn=10):
      if '_NOUN' in i:
        pass

def random_permutation(tt):
  for i in range(60):
    shift = 1
    n1 = randint(shift,len(tt)-1)
    if tt[n1] in russian_stopwords or tt[n1] in trash or tt[n1] == '[SSS]':
      continue
    tt[n1-shift],tt[n1] = tt[n1],tt[n1-shift]
  return tt

def random_del(tm):
  for i in range(25):
    n1 = randint(0,len(tm)-1)
    if tm[n1] in russian_stopwords or tm[n1] in trash or tm[n1] == '[SSS]':
      continue
    tm[n1] = ''
  return tm

def synonyms(t):
  for i in range(10):
    n1 = randint(1,len(t)-1)
    if t[n1] in russian_stopwords or t[n1] in trash:
      continue
    for tr in trash:
      t[n1] = t[n1].replace(tr,'')
    word = morph.parse(t[n1])[0][2]
    word,tag = tags_in_model(word)
    if not word:
      continue
    for i in model.most_similar(positive=[word], topn=10):
      if tag in i[0]:
        t[n1] = i[0].split('_')[0]
        break
  return t

def translate(text):
  translator = Translator()
  result = translator.translate(text,'en')
  return translator.translate(result.text,'ru').text

In [ ]:

from random import randint
X_aug_train = []
y_aug_train = []
for i in tqdm(range(len(X_train))):
  X_aug_train.append([X_train[i][0]])
  tx = X_train[i][0].split('[QRY]')[0]
  question = X_train[i][0].split('[QRY]')[1]
  t = tx.split()
  t1 = t.copy()
  permutation = ' '.join(random_permutation(t))
  word_delete = ' '.join(random_del(t1))
  X_aug_train.append([permutation + ' [QRY]' + question])
  X_aug_train.append([word_delete + ' [QRY]' + question])
  X_aug_train.append([translate(tx) + ' [QRY]' + question])
  y_aug_train.append(y_train[i])
  y_aug_train.append(y_train[i])
  y_aug_train.append(y_train[i])
  y_aug_train.append(y_train[i])

In [ ]:
X_aug_valid = []

for i in tqdm(range(len(X_val))):
  X_aug_valid.append([X_val[i][0]])
  question = X_val[i][0].split('[QRY]')[1]
  t = X_train[i][0].split('[QRY]')[0].split()
  t = random_permutation(t)
  t = synonyms(t)
  X_aug_valid.append([' '.join(t) + ' [QRY]' + question])


In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

In [ ]:
russian_stopwords = nltk.corpus.stopwords.words("russian")
trash = (",", "!", '"', "'", "<", ">", "«", "»", '...', '—', '&', '?', '.','»')

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize 

words = ' '.join(['день', 'ночь', 'человек', 'семантика', 'студент','человеческое','смешной'])
tokenized = sent_tokenize(words,'russian')
r = word_tokenize(tokenized[0])
print(r)
nltk.pos_tag(['день', 'ночь', 'человек', 'семантика', 'студент','человеческий','белый'])

['день', 'ночь', 'человек', 'семантика', 'студент', 'человеческое', 'смешной']


[('день', 'JJ'),
 ('ночь', 'NNP'),
 ('человек', 'NNP'),
 ('семантика', 'NNP'),
 ('студент', 'NNP'),
 ('человеческий', 'NNP'),
 ('белый', 'NN')]

In [ ]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'студент_NOUN', 'студент_ADJ','человеческий_ADJ','человеческий_NOUN','человек_ADJ','быстро_ADV']
for word in words:
    if word in model:
        print(word)
        for i in model.most_similar(positive=[word], topn=10):
            print(i[0], i[1])
        print('\n')
    else:
        print(word + ' is not present in the model')

# GPU

In [ ]:
if torch.cuda.is_available():        
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


# Токен и датасеты

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
tokenizer.add_special_tokens({'pad_token': '<pad>'})

In [ ]:
maxl = int(905)
batch_size = 5
X_train  = [tokenizer.encode(q[0], max_length=maxl, padding='max_length', truncation=True, pad_to_max_length=True,add_special_tokens=True) for q in tqdm(X_train)]
X_train = [i if i else [0] * maxl for i in X_train]
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
train_data = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(
    train_data,
    sampler=RandomSampler(train_data),
    batch_size=batch_size,
)

X_val  = [tokenizer.encode(q[0], max_length=maxl, padding='max_length', truncation=True, pad_to_max_length=True,add_special_tokens=True) for q in tqdm(X_val)]
X_val = [i if i else [0] * maxl for i in X_val]
X_val = torch.tensor(X_val)
y_val = torch.tensor(y_val)
validation_data = TensorDataset(X_val, y_val)
validation_dataloader = DataLoader(
    validation_data,
    sampler=SequentialSampler(validation_data),
    batch_size=batch_size,
    num_workers=4,
    pin_memory=True
)

In [ ]:
X_test  = [tokenizer.encode(q[0], max_length=maxl, padding='max_length', truncation=True, pad_to_max_length=True) for q in tqdm(X_test)]
X_test = [i if i else [0] * maxl for i in X_test]
X_test = torch.tensor(X_test)
y_test = torch.tensor([0]*(len(X_test)))
testing_data = TensorDataset(X_test, y_test)
testing_data = DataLoader(
    testing_data,
    sampler=SequentialSampler(testing_data),
    batch_size=batch_size,
    num_workers=4,
    pin_memory=True
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2', num_labels=2)
model.to(device)
model.config.pad_token_id = 0

Some weights of the model checkpoint at sberbank-ai/rugpt3small_based_on_gpt2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/rugpt3small_based_on_gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-6)


In [ ]:
del param_optimizer, optimizer_grouped_parameters, optimizer
torch.cuda.empty_cache()

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

epochs = 10

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

# Обучение

In [ ]:
import torch.nn as nn
quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.Embedding,nn.LSTM, nn.Linear}, dtype=torch.quint8
)


In [ ]:
from torch.quantization import convert
# model_ft_tuned.cpu()

model_quantized_and_trained = convert(model, inplace=False)

In [ ]:
import os
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 551.326499
Size (MB): 551.323023


In [ ]:
for _ in range(epochs):

    model.train()

    train_loss = 0
    pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))

    for step, batch in pbar:

        batch = (t.to(device) for t in batch)
        b_input_ids, b_labels = batch

        del batch
        torch.cuda.empty_cache()
        
        optimizer.zero_grad()

        loss = model(b_input_ids.long(), token_type_ids=None, labels=b_labels)
        loss[0].backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        train_loss += loss[0].item()

        pbar.set_description("loss %s" % loss[0].item())

        del loss, b_input_ids, b_labels
        torch.cuda.empty_cache()

    print("Loss на обучающей выборке: {0:.5f}".format(train_loss / len(train_dataloader)))
    
    model.eval()

    valid_preds, valid_labels = [], []

    for batch in tqdm(validation_dataloader): 

        batch = (t.to(device) for t in batch)
        b_input_ids, b_labels = batch

        with torch.no_grad():
            logits = model(b_input_ids.long(), token_type_ids=None)

        logits = logits[0].detach().cpu()
        label_ids = b_labels.to('cpu').numpy()

        batch_preds = torch.softmax(logits, axis=1).numpy()
        batch_labels = label_ids

        valid_preds.extend(batch_preds)
        valid_labels.extend(batch_labels)

    valid_preds = np.array(valid_preds)
    y_pred = []

    for i in valid_preds:
      if i[0]>i[1]:
        y_pred.append(0)
      else:
        y_pred.append(1)
        
    print("roc-auc: " + str(roc_auc_score(valid_labels, y_pred)))





Loss на обучающей выборке: 0.63471



roc-auc: 0.7076151417409791



Loss на обучающей выборке: 0.57105



roc-auc: 0.7434906665499074



Loss на обучающей выборке: 0.53892


...


roc-auc: 0.7635246240592358



Loss на обучающей выборке: 0.46457



roc-auc: 0.7644270764919655



Loss на обучающей выборке: 0.45643



roc-auc: 0.7618121536747571


In [ ]:
 y_pred_test = []
 for batch in tqdm(testing_data): 
   
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_labels = batch

        with torch.no_grad():
            logits = model(b_input_ids.long(), token_type_ids=None)

        logits = logits[0].detach().cpu()
        label_ids = b_labels.to('cpu').numpy()

        batch_preds = torch.softmax(logits, axis=1).numpy()
        y_pred_test.append(batch_preds)

In [ ]:
y_pred_test_result = []
for j in y_pred_test:
  for i in j:
    if i[0]>i[1]:
      y_pred_test_result.append(0)
    else:
      y_pred_test_result.append(1)

In [ ]:
df1 = y_pred_test_result
c = 0
with open('test.jsonl', 'r') as json_file:
        json_list = list(json_file)
        
new_test = [json.loads(item) for item in json_list]

def get_answer(dic):
  global c
  dic['label'] = int(df1[c])
  c += 1
  return dic

for i in range(len(new_test)):
    q = new_test[i]['passage']['questions']
    for j in range(len(q)):
        q[j]['answers'] = [get_answer(dic) for dic in q[j]['answers'] ]
    new_test[i]['passage']['questions'] = q

In [ ]:
import jsonlines
with jsonlines.open('predict.jsonl', 'w') as writer:
    writer.write_all(new_test)

In [ ]:
from google.colab import files
files.download('predict.jsonl')